In [2]:
import numpy as np
import networkx as nx

In [3]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [4]:
def string_to_grid(s, map_func=int):
    return [list(map(map_func, list(x))) for x in s.split("\n")]

In [5]:
def bi_range(a, b):
    if(a > b):
        return range(a, b-1, -1)
    else:
        return range(a, b+1)

In [6]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [7]:
class Point3D:
    def __init__(self, coords):
        self.coords = tuple(coords)
        self.x = coords[0]
        self.y = coords[1]
        self.z = coords[2]

    def __sub__(self, other):
        return Point3D(((self.x - other.x), (self.y - other.y), (self.z - other.z)))
    
    def __add__(self, other):
        return Point3D(((self.x + other.x), (self.y + other.y), (self.z + other.z)))
    
    def __eq__(self, other):
        return self.coords == other.coords
    
    def __array__(self):
        return np.asarray(self.coords)
    
    def __hash__(self):
        return hash(self.coords)
        
    def s_dist(self, other):
        return sum(map(lambda x: x **2, (self - other).coords))

    def m_dist(self, other):
        return sum(map(abs, (self - other).coords))

class Point2D:
    def __init__(self, coords):
        self.coords = tuple(coords)
        self.x = coords[0]
        self.y = coords[1]

    def __sub__(self, other):
        return Point2D(((self.x - other.x), (self.y - other.y)))
    
    def __add__(self, other):
        return Point2D(((self.x + other.x), (self.y + other.y)))
    
    def __eq__(self, other):
        return self.coords == other.coords
    
    def scale(self, c):
        return Point2D(((self.x * c), (self.y *c)))
    
    def __array__(self):
        return np.asarray(self.coords)
    
    def __hash__(self):
        return hash(self.coords)
        
    def s_dist(self, other):
        return sum(map(lambda x: x **2, (self - other).coords))

    def m_dist(self, other):
        return abs(self.x - other.x) + abs(self.y - other.y)
    
    def __str__(self):
        return str(self.coords)
    
    def __repr__(self):
        return str(self.coords)
    
    def __iter__(self):
        for c in self.coords:
            yield c

In [9]:
directions = [
    Point2D((0, 1)),
    Point2D((0, -1)),
    Point2D((1, 0)),
    Point2D((-1, 0))
]

class Grid(np.ndarray):
    def __new__(cls, grid_str, conv_func=int):
        grid = [[conv_func(p) for p in row] for row in grid_str.split("\n")]
        obj = np.asarray(grid).view(cls)
        return obj

    def print(self):
        print("\n".join(["".join(map(str,row)) for row in self]))
        
    def in_bounds(self, p):
        row, col = p
        return (row >= 0 and row < len(self) and col >= 0 and col < len(self[0]))
    
    def sides(self, p, vals=False, both=False):
        row, col = p
        side_list = []
        if(row != 0):
            side_list.append((row-1, col))
        if(col != 0):
            side_list.append((row, col-1))
        if(row != len(self) - 1):
            side_list.append((row+1, col))
        if(col != len(self[0]) - 1):
            side_list.append((row, col+1))
        if(vals or both):
            vals_side_list = [self[point[0]][point[1]] for point in side_list]
            if vals:
                return vals_side_list
            if both:
                return list(zip(side_list, vals_side_list))
        return side_list
    
    def diags(self, p, vals=False):
        row, col = p
        side_list = []
        if(row != 0 and col != 0):
            side_list.append((row-1, col-1))
        if(col != 0 and row != len(self) - 1):
            side_list.append((row+1, col-1))
        if(row != len(self) - 1 and  col != len(self[0]) - 1):
            side_list.append((row+1, col+1))
        if(col != len(self[0]) - 1 and row != 0):
            side_list.append((row-1, col+1))
        if(vals):
            side_list = [self[point[0]][point[1]] for point in side_list]
        return side_list
    
    def all_sides(self, p, vals=False):
        return self.sides(p, vals=vals) + self.diags(p, vals=vals)
    
    def to_graph(self, valid=None, invalid=None):
        graph = nx.Graph()
        for row in range(len(self)):
            for col in range(len(self[0])):
                if ((valid is not None and self[row][col] in valid) or
                    (invalid is not None and self[row][col] not in invalid)):
                    graph.add_node((row, col), val=self[row][col])
                    if(row != 0):
                        if ((valid is not None and self[row-1][col] in valid) or
                            (invalid is not None and self[row-1][col] not in invalid)):
                            graph.add_edge((row, col), (row-1, col))
                    if(col != 0):
                        if ((valid is not None and self[row][col-1] in valid) or
                            (invalid is not None and self[row][col-1] not in invalid)):
                            graph.add_edge((row, col), (row, col-1))
                    if(row != len(self) - 1):
                        if ((valid is not None and self[row+1][col] in valid) or
                            (invalid is not None and self[row+1][col] not in invalid)):
                            graph.add_edge((row, col), (row+1, col))
                    if(col != len(self[0]) - 1):
                        if ((valid is not None and self[row][col+1] in valid) or
                            (invalid is not None and self[row][col+1] not in invalid)):
                            graph.add_edge((row, col), (row, col+1))
        return graph